<a href="https://colab.research.google.com/github/Alaa-alsulaiman/Econometric-Example/blob/main/alaa_alsulaiman__assignment_w5_c1_template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Week 5, Assignment C1:

Logistic Regression with Yahoo! Finance API in Python

In this notebook, we will study Logistic Regression using the Yahoo! Finance API in Python.

Please complete the lines bellow where you see "#TODO".

In [1]:
! pip install yfinance

In [2]:
#Objective 1.
#Use this cell to import the Numpy (as np), Pandas (as pd), and YFinance (as yf) packages.
import pandas
import numpy as np
import pandas as pd
import yfinance as yf
#Our work will also require some components of the Sklearn and Pandas_Datareader packages as imported below:

import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from pandas_datareader import data as pdr
yf.pdr_override()

In [3]:
#Objective 2.
#Select a stock symbol for a stock whose historical data is available on the Yahoo! Finance website.  Store the
#string of your chosen symbol to the new variable "stock_symbol".

stock_symbol = 'AMZN'

In [4]:
#Objective 3.
#Use the function "pdr.get_data_yahoo(stock_symbol, start_date, end_date)" to generate a Pandas dataframe of
#historical stock data for your chosen stock.  Retain only the first four columns of the dataframe.  Be sure to 
#drop any rows containing NaN's, and take a peek at the resulting dataframe to make sure everything looks good.

df = pdr.get_data_yahoo(stock_symbol, pandas.Timestamp('2019'), pandas.Timestamp('2021'))
df = df[['Open','High','Low','Close']]
df = df.dropna()

[*********************100%***********************]  1 of 1 completed


In [5]:
df[pdr.get_data_yahoo(stock_symbol, pandas.Timestamp('2019'), pandas.Timestamp('2021')).columns[0:4]]

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close
Date,,,,
2019-01-02,1465.199951,1553.359985,1460.930054,1539.130005
2019-01-03,1520.010010,1538.000000,1497.109985,1500.280029
2019-01-04,1530.000000,1594.000000,1518.310059,1575.390015
2019-01-07,1602.310059,1634.560059,1589.189941,1629.510010
2019-01-08,1664.689941,1676.609985,1616.609985,1656.579956
...,...,...,...,...
2020-12-24,3193.899902,3202.000000,3169.000000,3172.689941
2020-12-28,3194.000000,3304.000000,3172.689941,3283.959961
2020-12-29,3309.939941,3350.649902,3281.219971,3322.000000


In [6]:
#Objective 4.
#Use the historical data to define some predictor variables.  Add these variables to the dataframe.  Include, at a
#minimum, (Predictor.I) the rolling average closing price over the past fifteen (15) days and (Predictor.II) the
#change in opening price over the past one (1) day.  Be sure to again drop any rows containing NaN's, and take a
#peek at the resulting dataframe to make sure everything looks good.  Store the dataframe as the new variable "X".

df['Close_15_Rolling'] = df['Close'].rolling(window=15).mean()
df['Open_1_Change'] = df['Open'].diff()
df = df.dropna()
X = df

In [7]:
#Objective 5.
#Define the target or dependent variable to be one (1) if the change in closing price over the past one (1) day is
#nonnegative and negative one (-1) if the change in closing price over the past one (1) day is negative.  This
#variable's values should be forward-looking (i.e., you should subtract today's price from tomorrow's price rather
#than subtracting yesterday's price from today's price).  Store the resulting values as the new variable "y".  You
#may find the "np.where(*args)" function to be useful.  Look it up in Numpy documentation for support.

y = np.where((df.Close.shift(-1)-df['Close'])>=0,1,-1)

In [8]:
#Objective 6.
#Split the data into training and test sets, putting the first seventy percent (70%) of the data in the training
#set.

index = int(0.7*len(X))
X_train= X[:index]
X_test= X[index:]
y_train= y[:index]
y_test = y[index:]

In [9]:
#Objective 7.
#Instantiate the Logistic Regression model object, and use its ".fit(*args)" method to fit the model to the 
#training data.

logistic = LogisticRegression()
logistic = logistic.fit(X_train,y_train)

In [10]:
#Objective 8.
#Examine the model's coefficients by using its ".coef_" method.

pd.DataFrame({'column':X.columns.values, 'criteia': logistic.coef_.ravel()})

,column,criteia
0,Open,-0.009190
1,High,0.017672
2,Low,-0.001976
3,Close,-0.005930
4,Close_15_Rolling,-0.000662
5,Open_1_Change,-0.006448


In [11]:
#Objective 9.
#Use the model's ".predict_proba(*args)" and ".predict(*args)" methods to generate predictions over the test set.

probabilities = logistic.predict_proba(X_test)
predictions = logistic.predict(X_test)

In [12]:
#Objective 10.
#Use the function "metrics.confusion_matrix(*args)" to create a confusion matrix comparing the predicted and true
#classification labels over the test set.

sklearn.metrics.confusion_matrix(y_pred=predictions,y_true=y_test)

array([[15, 54],
       [16, 63]])

In [13]:
print(sklearn.metrics.classification_report(y_pred=predictions,y_true=y_test))

              precision    recall  f1-score   support

          -1       0.48      0.22      0.30        69
           1       0.54      0.80      0.64        79

    accuracy                           0.53       148
   macro avg       0.51      0.51      0.47       148
weighted avg       0.51      0.53      0.48       148



In [14]:
#Objective 11.
#Calculate the model's accuracy on the test set using its ".score(*args)" method.

logistic.score(X_test,y_test)

0.527027027027027

In [30]:
#Objective 12.
#Use five-fold cross validation to cross-check the accuracy of the model over different held-out test sets.  This
#is where you should use the function "cross_val_score(*args)".

cross_val = cross_val_score(logistic,X,y, cv=5)
cross_val

array([0.56565657, 0.52040816, 0.55102041, 0.51020408, 0.58163265])